You are a Python Developer. Build a simple command line style tool in Python to work with CSV files in the first cell of copilot.ipynb

Features:
- `--convert` : Convert CSV to JSON
- `--print`   : Pretty print as JSON
- `--table`   : Display as ASCII table
- Later: add filtering, summaries

In [5]:
# Simple CLI tool for CSV files: --convert, --print, --table
import argparse
import csv
import json
from tabulate import tabulate
import sys

def read_csv(file_path):
    with open(file_path, newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        return list(reader)

def convert_to_json(data, out_path=None):
    json_str = json.dumps(data, indent=2, ensure_ascii=False)
    if out_path:
        with open(out_path, 'w', encoding='utf-8') as f:
            f.write(json_str)
    else:
        print(json_str)

def pretty_print_json(data):
    print(json.dumps(data, indent=2, ensure_ascii=False))

def print_table(data):
    if not data:
        print('No data to display.')
        return
    print(tabulate(data, headers='keys', tablefmt='grid'))

def main():
    parser = argparse.ArgumentParser(description='CSV Utility Tool')
    parser.add_argument('csvfile', help='Path to CSV file')
    parser.add_argument('--convert', metavar='OUT', help='Convert CSV to JSON and save to OUT')
    parser.add_argument('--print', action='store_true', help='Pretty print CSV as JSON')
    parser.add_argument('--table', action='store_true', help='Display CSV as ASCII table')
    args = parser.parse_args()

    data = read_csv(args.csvfile)

    if args.convert:
        convert_to_json(data, args.convert)
    if args.print:
        pretty_print_json(data)
    if args.table:
        print_table(data)
    if not (args.convert or args.print or args.table):
        parser.print_help()
        
# To use in notebook, you can call main() with sys.argv set, or copy this to a .py file and run from command line.
# Example usage:
#   python CLIUtility.py data.csv --convert out.json
#   python CLIUtility.py data.csv --print
#   python CLIUtility.py data.csv --table

Create a small CSV file called data.csv with columns: name, age, city.
Include 5 rows of realistic data.


In [7]:
if __name__ == '__main__':
    sys.argv = ['script.py', 'data.csv', '--table']
    main()


+---------+-------+---------------+
| name    |   age | city          |
+=========+=======+===============+
| Alice   |    30 | New York      |
+---------+-------+---------------+
| Bob     |    25 | San Francisco |
+---------+-------+---------------+
| Charlie |    35 | Chicago       |
+---------+-------+---------------+
| Diana   |    28 | Seattle       |
+---------+-------+---------------+
| Ethan   |    40 | Boston        |
+---------+-------+---------------+


Write a function csvutil(argv) that parses arguments like:
--convert, --print, --table
and just prints which option was selected.
Use argparse.

Update csvutil so that with --convert it reads the CSV file 
and writes a JSON file with the same name but .json extension.

In [22]:
def csvutil(argv):
    import argparse
    import csv
    import json
    import os
    parser = argparse.ArgumentParser(description='CSV Utility Demo')
    parser.add_argument('csvfile', help='Path to CSV file')
    parser.add_argument('--convert', action='store_true', help='Convert CSV to JSON and save to .json file')
    parser.add_argument('--print', action='store_true', help='Pretty print CSV as JSON')
    parser.add_argument('--table', action='store_true', help='Display CSV as ASCII table')
    parser.add_argument('--filter', metavar='COLUMN=VALUE', type=str, help='Only include rows where COLUMN equals VALUE')
    parser.add_argument('--summary', action='store_true', help='Print min, max, and average for numeric columns')
    args = parser.parse_args(argv)

    # Normalize filter argument to support both '--filter=city=Boston' and '--filter', 'city=Boston'
    filter_value = None
    for i, arg in enumerate(argv):
        if arg.startswith('--filter='):
            filter_value = arg.split('=', 1)[1]
        elif arg == '--filter' and i+1 < len(argv):
            filter_value = argv[i+1]
    if filter_value is None:
        filter_value = args.filter

    # Always read the CSV if any action is requested
    data = None
    if args.convert or args.print or args.table or args.summary:
        with open(args.csvfile, newline='', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            data = list(reader)

    # Apply filter if specified
    if filter_value and data is not None:
        if '=' in filter_value:
            col, val = filter_value.split('=', 1)
            data = [row for row in data if row.get(col) == val]
        else:
            print('Invalid --filter format. Use --filter column=value')
            return

    if args.convert:
        json_path = os.path.splitext(args.csvfile)[0] + '.json'
        with open(json_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=2, ensure_ascii=False)
        print(f'Converted {args.csvfile} to {json_path}')
    if args.print:
        print(json.dumps(data, indent=2, ensure_ascii=False))
    if args.table:
        from tabulate import tabulate
        if data:
            print(tabulate(data, headers='keys', tablefmt='grid'))
        else:
            print('No data to display.')
    if args.summary:
        if not data:
            print('No data to summarize.')
            return
        numeric_cols = []
        # Find numeric columns
        for key in data[0].keys():
            try:
                float(data[0][key])
                numeric_cols.append(key)
            except (ValueError, TypeError):
                continue
        if not numeric_cols:
            print('No numeric columns found.')
            return
        print('Summary statistics:')
        for col in numeric_cols:
            values = []
            for row in data:
                try:
                    values.append(float(row[col]))
                except (ValueError, TypeError):
                    continue
            if values:
                print(f"{col}: min={min(values)}, max={max(values)}, avg={sum(values)/len(values):.2f}")
            else:
                print(f"{col}: No valid numeric data.")
    if not (args.convert or args.print or args.table or args.summary):
        print('No option selected.')

In [18]:
csvutil(['data.csv', '--convert'])

Converted data.csv to data.json


Add support for --print to pretty print the CSV data as JSON to stdout.


In [19]:
csvutil(['data.csv', '--print'])

[
  {
    "name": "Alice",
    "age": "30",
    "city": "New York"
  },
  {
    "name": "Bob",
    "age": "25",
    "city": "San Francisco"
  },
  {
    "name": "Charlie",
    "age": "35",
    "city": "Chicago"
  },
  {
    "name": "Diana",
    "age": "28",
    "city": "Seattle"
  },
  {
    "name": "Ethan",
    "age": "40",
    "city": "Boston"
  }
]


Add optional argument --filter column=value so that only rows matching that filter are included.

In [20]:
# Test csvutil with --filter argument (should work with both forms)
print('Test with --filter=city=Boston')
csvutil(['data.csv', '--print', '--filter=city=Boston'])
print('\nTest with --filter city=Boston')
csvutil(['data.csv', '--print', '--filter', 'city=Boston'])

Test with --filter=city=Boston
[
  {
    "name": "Ethan",
    "age": "40",
    "city": "Boston"
  }
]

Test with --filter city=Boston
[
  {
    "name": "Ethan",
    "age": "40",
    "city": "Boston"
  }
]


Add a --summary option that prints min, max, and average for numeric columns.


In [23]:
csvutil(['data.csv', '--summary'])

Summary statistics:
age: min=25.0, max=40.0, avg=31.60
